In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import json

In [2]:
data = list(map(lambda k: list(map(float, k.split())), open("zipcombo.dat").read().split("\n")[:-1]))

In [3]:
def plot_digit(data):
    label, *pixels = data
    pixels = np.array(pixels, dtype='uint8')
    pixels = pixels.reshape((16, 16))
    plt.title('Label is {label}'.format(label=label))
    plt.imshow(pixels, cmap='gray')
    plt.show()

In [4]:
kernel_dot = lambda a, b, r: (a.dot(b))**r

In [5]:
def get_classifier(data_len, num_classes):
    """
    Initialise a classifier of shape [data_len, num_classes]
    """
    return [[0] * data_len for _ in range(num_classes)]

In [6]:
def predict(class_vector, kernel_vector):
    """
    Use the given kernel to create a prediction vector.
    """
    return sum([classifier[i] * kernel(datapoint[1:], pattern, d) for i, datapoint in enumerate(data)])

In [15]:
sign = lambda k: -1 if k <= 0 else 1
def train(data, classifier, dot_matrix):
    mistakes = 0
    for i, datapoint in enumerate(data):
        label, *pattern = datapoint
        pred_vector = [sum(classifier[j]*dot_matrix[i]) for j in range(10)]
        maxima = -float('inf')
        max_i = 0
        for j in range(10):
            y = 1 if label == j else -1
            if y * pred_vector[j] <= 0:
                classifier[j][i] -= sign(pred_vector[j])
            if pred_vector[j] > maxima:
                maxima = pred_vector[j]
                max_i = j
        if max_i != label:
            mistakes += 1
    return classifier, (mistakes / len(data)) * 100            

In [27]:
def test(data, classifier, dot_matrix, l):
    mistakes = 0
    for i, datapoint in enumerate(data):
        label, *pattern = datapoint
        pred_vector = [sum(classifier[j] * dot_matrix[l + i]) for j in range(10)]
        maxima, max_i = -float('inf'), 0
        for j in range(10):
            if pred_vector[j] > maxima:
                maxima, max_i = pred_vector[j], j
        if max_i != label:
            mistakes += 1
    return (mistakes / len(data)) * 100

In [40]:
from sklearn.model_selection import train_test_split
def prep_training(train_set, test_set, d, epoch):
    np_data = np.asarray(train_set)
    classifier = get_classifier(len(train_set), 10)
    # labels
    labels = np_data[:, :1]
    np_data = np_data[:, 1:] # delete the labels

    dot_matrix = np_data @ np_data.T
    dot_matrix = dot_matrix**d
    
    for i in range(epoch):
        classifier, mistakes = train(train_set, classifier, dot_matrix)
        print("epoch {}: Training error: {:0.2f}%".format(i, mistakes))
    
    # we need dot product for the test set. So append the test set with train set, 
    # apply the same matrix trick. Then index it by, i + len(train_set)
    test_classifier = np.append(classifier, np.array([[0] * len(test_set) for _ in range(10)]), axis=1)
    train_len = len(train_set)
    train_set.extend(test_set)
    np_test_data = np.asarray(train_set)
    
    np_test_data = np_test_data[:, 1:] # delete the labels
    dot_matrix = np_test_data @ np_test_data.T
    dot_matrix = dot_matrix**d

    mistakes = test(test_set, test_classifier, dot_matrix, train_len)
    print("Test error: {:0.2f}".format(mistakes))

In [42]:
train_set, test_set = train_test_split(data, test_size=0.2)
prep_training(train_set, test_set, 3, 5)

580 7438
epoch 0: Training error: 7.797795106211347%
128 7438
epoch 1: Training error: 1.720892713094918%
67 7438
epoch 2: Training error: 0.9007797795106212%
28 7438
epoch 3: Training error: 0.3764452809895133%
21 7438
epoch 4: Training error: 0.282333960742135%
[[-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 ...
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]]
55 1860
Test error: 2.956989247311828
